In [2]:
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import KFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier #support from multiclass

from sklearn.svm import SVC


sc = StandardScaler()

In [3]:


data_dir = os.getenv('FINANCE_DATA')
data_only_drive = '/mnt/usb-Seagate_Expansion_Desk_NA8XEHR6-0:0-part2' #external date only drive

ftse_symbols = [s for s in os.listdir(data_dir) if s.endswith('.L')]
features_models = os.path.join(data_dir, 'features_models')
labels = os.path.join(features_models, 'labels')
features = os.path.join(features_models, 'features')
hmm_models = os.path.join(features_models, 'models', 'HMM')

os.listdir(os.path.join(features,ftse_symbols[1],'MODEL_BASED')) #list of all the model -dates

['20171006',
 '20170922',
 '20180405',
 '20180227',
 '20170711',
 '20170216',
 '20171020',
 '20170904',
 '20180418',
 '20170824',
 '20180404',
 '20170905',
 '20170703',
 '20170918',
 '20180219',
 '20180419',
 '20170914',
 '20171002',
 '20170915',
 '20170214',
 '20180412',
 '20170829',
 '20170817',
 '20170202',
 '20171024',
 '20170719',
 '20170809',
 '20180207',
 '20170807',
 '20180403',
 '20171010',
 '20170727',
 '20170206',
 '20170912',
 '20170821',
 '20180221',
 '20180209',
 '20170823',
 '20170814',
 '20170811',
 '20170704',
 '20170228',
 '20170818',
 '20170203',
 '20170223',
 '20170816',
 '20170215',
 '20180406',
 '20170717',
 '20170830',
 '20171027',
 '20170705',
 '20180205',
 '20180208',
 '20170927',
 '20170710',
 '20170731',
 '20171018',
 '20170803',
 '20170928',
 '20170804',
 '20180413',
 '20170227',
 '20180409',
 '20171017',
 '20170911',
 '20170201',
 '20180216',
 '20170210',
 '20170209',
 '20171013',
 '20180214',
 '20170831',
 '20170913',
 '20171019',
 '20171005',
 '20170217',

In [4]:
symbol_labels_path= os.path.join(labels, ftse_symbols[1])
# dates_features =os.listdir(os.path.join(features, ftse_symbols[1],'MODEL_BASED'))
# test= os.path.join(symbol_features_path, dates_features[1])
# features_for_date=os.listdir(test)
# "_".join((features_for_date[1].split('_')[0],features_for_date[1].split('_')[1],features_for_date[1].split('_')[2]))

In [19]:
symbol_features_path=os.path.join(features, ftse_symbols[1],'MODEL_BASED')
dates_for_symbol

['20170928.csv',
 '20171027.csv',
 '20170803.csv',
 '20170711.csv',
 '20170824.csv',
 '20170808.csv',
 '20170210.csv',
 '20171004.csv',
 '20170705.csv',
 '20171024.csv',
 '20180220.csv',
 '20170904.csv',
 '20170224.csv',
 '20170915.csv',
 '20170913.csv',
 '20171020.csv',
 '20170802.csv',
 '20171009.csv',
 '20170830.csv',
 '20170223.csv',
 '20171003.csv',
 '20170810.csv',
 '20170727.csv',
 '20170213.csv',
 '20170712.csv',
 '20170831.csv',
 '20170829.csv',
 '20180411.csv',
 '20170801.csv',
 '20170704.csv',
 '20170901.csv',
 '20171018.csv',
 '20170821.csv',
 '20180201.csv',
 '20170908.csv',
 '20170228.csv',
 '20170914.csv',
 '20180221.csv',
 '20170725.csv',
 '20170728.csv',
 '20170227.csv',
 '20170222.csv',
 '20171026.csv',
 '20180205.csv',
 '20170922.csv',
 '20170209.csv',
 '20180419.csv',
 '20180405.csv',
 '20170703.csv',
 '20170825.csv',
 '20170207.csv',
 '20180212.csv',
 '20170719.csv',
 '20171010.csv',
 '20170221.csv',
 '20170816.csv',
 '20180417.csv',
 '20180219.csv',
 '20170815.csv

In [6]:
#####OOP#####
class DataLoader(object):
    def __init__(self, path_main, ticker):
        self.main_path = path_main
        self.ticker = ticker
        
        self.features_labels_path = os.path.join(self.main_path, 'features_models')
        self.features_path = os.path.join(self.features_labels_path,'features')
        # collection of per symbol non directional labels
        self.labels_path = os.path.join(self.features_labels_path, 'labels', self.ticker, 'NON_DIRECTIONAL')
        self.symbol_features_path = os.path.join(self.features_labels_path,'features', self.ticker, 'MODEL_BASED')
         #list of all the model -oos hmm feature dates - each folder is a collection of oos feature dates
        self.oos_dates_list = os.listdir(self.symbol_features_path)
        
    def ticker_features(self, model_date, date):
        #need to make this a lot more flexible with number of states
        if model_date < date:
            file_name = "_".join((self.ticker, '3', 'states','features','date:',date,'now:', '20181224','.pickle'))
            file_loc = os.path.join(self.symbol_features_path,str(model_date),file_name)
            with open(file_loc, 'rb') as handle:
                ticker_features = pickle.load(handle)
        else:
            print('Loading Feature Date which is in-sample. Change your Model Date')
        return ticker_features

    def ticker_labels_csv(self, date):
        file_loc = os.path.join(self.labels_path, str(date)+'.csv')
        ticker_labels = pd.read_csv(file_loc, index_col=0)
        return ticker_labels

    @staticmethod
    def open_pickle_file(path, pickle_file):
        file_loc = os.path.join(path, pickle_file)
        pickle_to_file = pickle.load(open(file_loc, "rb"))
        return pickle_to_file

    @staticmethod
    def get_date_from_file(file_, numb_):
        return os.path.splitext(file_[numb_])[0]


class MarketFeatures(object):
    # a class to be expanded that uses features for base case -market based only-indicators/features
    """"Requires:
    a dataframe that has TradedPrice And Volume columns
    symbol - A stock symbol on which to form a strategy on.
    short_window - Lookback period for short moving average.
    long_window - Lookback period for long moving average.
    """

    def __init__(self, df):
#         self.ticker = ticker
        self.df = df

    def load_data(self):
        pass

    def ma_spread(self, short_window=5, long_window=20):
        # function that produces the MA spread, which can be used on its own or as an input for MACD
        short_rolling_px = self.df['TradedPrice'].rolling(window=short_window).mean()
        long_rolling_px = self.df['TradedPrice'].rolling(window=long_window).mean()
        px_name = "_".join(('px_indx', str(short_window), str(long_window)))
        self.df[px_name] = long_rolling_px - short_rolling_px
        return self.df

    def obv_calc(self):
        # on balance volume indicator
        self.df['SignedVolume'] = self.df['Volume'] * np.sign(self.df['TradedPrice'].diff()).cumsum()
        self.df['SignedVolume'].iat[1] = 0
        self.df['OBV'] = self.df['SignedVolume'] #.cumsum()
        self.df = df.drop(columns=['SignedVolume'])
        return self.df

    def chaikin_mf(self, period=5):
        # chaiking money flow indicator
        self.df["MF Multiplier"] = (self.df['TradedPrice'] - (self.df['TradedPrice'].expanding(period).min()) \
                               - (self.df['TradedPrice'].expanding(period).max() \
                                  - self.df['TradedPrice'])) / (
                                          self.df['TradedPrice'].expanding(period).max() - self.df[
                                      'TradedPrice'].expanding(period).min())
        self.df["MF Volume"] = self.df['MF Multiplier'] * df['Volume']
        self.df['CMF_' + str(period)] = self.df['MF Volume'].sum() / self.df["Volume"].rolling(period).sum()
        self.df = self.df.drop(columns=['MF Multiplier', 'MF Volume'])
        return self.df


class FitModels(object):

    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    # # Train a SVM classification model

    def best_kernel_ridge(self, kernel_choice):

        kr_clf =OneVsRestClassifier(GridSearchCV(KernelRidge(kernel=str(kernel_choice)), cv=10,
                          param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                                      "gamma": np.logspace(-2, 2, 5)})).fit(self.X_train, self.y_train)

        return kr_clf

    def best_svm_clf(self, kernel_choice):

        param_grid = dict(kernel=[str(kernel_choice)],
                          C=[1, 5, 10, 25, 50, 100],
                          gamma=[0.0001, 0.001, 0.01, 0.02, 0.05, 0.01])

        clf = OneVsRestClassifier(
            GridSearchCV(SVC(class_weight='balanced'), param_grid, verbose=1, n_jobs=-1, cv=10))\
            .fit(self.X_train, self.y_train)
        return clf

    def best_gradient_boost_clf(self):
        #this needs to be written properly- just a baseline placeholder here!
        GBR = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                           max_depth=4, max_features='sqrt',
                                           min_samples_leaf=15, min_samples_split=10,loss='huber',
                                        random_state=5)

        gb_boost_clf = OneVsRestClassifier(GBR).fit(self.X_train, self.y_train)

        return gb_boost_clf
    def best_MKL_clf(self):
        pass
    def best_knn_clf(self):
        pass
    def best_random_forest_clf(self):
        pass


    def run_cv(self, clf_class, **kwargs):
        # Construct a kfolds object
        kf = KFold(len(self.y_train), n_folds=10, shuffle=True)
        y_pred = self.y_train.copy()

        # Iterate through folds
        for train_index, test_index in kf:
            X_train_local, X_test_local = self.X_train[train_index], self.X_train[test_index]
            y_train_local = self.y_train[train_index]
            # Initialize a classifier with key word arguments
            clf = clf_class(**kwargs)
            clf.fit(self.X_train, self.y_train)
            y_pred[test_index] = clf.predict(X_test_local)
        return y_pred


class PredictModels(FitModels):
    def __init__(self):
        pass

def no_nans(label):
    return np.sum(np.isnan(label))


def remove_last_element(arr):
    return arr[np.arange(arr.size - 1)]


In [7]:
def fwd_dates(_dates_list, _key_date):
    #returns a list of dates that are forward from the key_date
    fwd_dates_list  = [i for i in _dates_list if i > _key_date]
    return fwd_dates_list

def prec_recall_report(y_true_, y_predict_):
    #function to ge the sci-kit learn classification metrics into a pretty DF for csv!
    report = pd.DataFrame(list(precision_recall_fscore_support(y_true_, y_predict_)),
                          index=['Precision', 'Recall', 'F1-score', 'Support']).T
    # Now add the 'Avg/Total' row
    report.loc['Avg/Total', :] = precision_recall_fscore_support(y_true_, y_predict_, average='weighted')
    report.loc['Avg/Total', 'Support'] = report['Support'].sum()
    return report

In [8]:

if __name__ == '__main__' :

    data_dir = os.getenv('FINANCE_DATA')
    data_only_drive = '/mnt/usb-Seagate_Expansion_Desk_NA8XEHR6-0:0-part2' #external date only drive

    ftse_symbols = [s for s in os.listdir(data_dir) if s.endswith('.L')]
    features_models = os.path.join(data_dir, 'features_models')
    labels = os.path.join(features_models, 'labels')
    features = os.path.join(features_models, 'features')
    hmm_models = os.path.join(features_models, 'models', 'HMM')
    model_save_loc = os.path.join(data_only_drive, 'Data','features_models','models')


In [9]:
test_date='20170928'
datacls = DataLoader(path_main=data_dir, ticker=ftse_symbols[1])
dates_for_symbol =os.listdir(datacls.labels_path)
df =datacls.ticker_labels_csv(date='20180215')
df_final =MarketFeatures(df=MarketFeatures(df=MarketFeatures(df=df).obv_calc()).chaikin_mf()).ma_spread()

/home/ak/virtualenvs/DataAnalysis/lib/python2.7/site-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
df_final['Volume'].iat[0]=100

In [18]:
df_final['Volume']

0        100.0
1       1000.0
2       2000.0
3       1556.0
4       1884.0
5       4314.0
6         30.0
7        757.0
8       2000.0
9        352.0
10      1956.0
11       315.0
12      3682.0
13       993.0
14      1808.0
15      2000.0
16      4621.0
17      1479.0
18      2077.0
19       853.0
20      3040.0
21       484.0
22      2502.0
23       539.0
24       525.0
25      2600.0
26      2412.0
27      1850.0
28      2800.0
29      1669.0
         ...  
1699     474.0
1700    2000.0
1701    1655.0
1702    2133.0
1703     554.0
1704     400.0
1705    1916.0
1706    2929.0
1707     193.0
1708    1010.0
1709    2946.0
1710     273.0
1711     299.0
1712    2488.0
1713     454.0
1714    1266.0
1715    3775.0
1716     248.0
1717     447.0
1718    2775.0
1719     127.0
1720    1600.0
1721    3300.0
1722    1306.0
1723     800.0
1724     800.0
1725     400.0
1726    1500.0
1727    4941.0
1728     577.0
Name: Volume, Length: 1729, dtype: float64

In [ ]:
model_dates = datacls.oos_dates_list #list of dates that hmm models have been fitted and used out of sample
model_date_idx = 1 #picking one such hmm model
#path for hmm model features
feature_files_path =os.path.join(symbol_features_path,model_dates[model_date_idx])
#get all the feature files for each model date
list_of_feature_files_for_model_date= os.listdir(os.path.join(symbol_features_path,model_dates[model_date_idx]))
#get the dates for those feature files
feature_dt_model_date=[list_of_feature_files_for_model_date[idx].split("_")[5] for idx,_ in enumerate(list_of_feature_files_for_model_date)]
#pick a feature file
test_pickle_file =list_of_feature_files_for_model_date[1]
#get all the features in one go
features_tuple = datacls.open_pickle_file(path= feature_files_path, pickle_file=test_pickle_file)
#get labels file
df =datacls.ticker_labels_csv(date=list_of_feature_files_for_model_date[1].split("_")[5])
df_w_market_features =MarketFeatures(df=MarketFeatures(df=MarketFeatures(df=df).obv_calc()).chaikin_mf()).ma_spread()


In [ ]:
features_df = pd.concat([features_tuple[0], features_tuple[1], features_tuple[2], \
                             features_tuple[3]], axis=1, sort=False)

df_concat = pd.concat([features_df, df_w_market_features], axis=1, sort='False').dropna()

In [ ]:
#right order of things

y=np.asanyarray(df_concat['Duration'])
df_final=df_concat.drop(columns=['TradedPrice','Duration','TradedTime'])
X = df_final

In [ ]:
print(X.shape, y.shape)

In [ ]:
from sklearn.svm import SVR
np.random.seed(0)
# svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
#                    param_grid={"C": [1e0, 1e1, 1e2, 1e3],
#                                "gamma": np.logspace(-2, 2, 5)})
# svr.fit(X,y)

In [ ]:
y_svr = svr.predict(X)

In [ ]:
import time
train_time = []
# #############################################################################
# Fit regression model
t0 = time.time()
# svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
# svr_poly = SVR(kernel='poly', C=1e3, degree=2)
# y_rbf = svr_rbf.fit(X, y).predict(X)
y_lin = svr_lin.fit(X, y).predict(X)
# y_poly = svr_poly.fit(X, y).predict(X)
test_time.append(time.time() - t0)

In [ ]:
all_features= datacls.ticker_features(date='20180215',model_date=test_date)
fisher = all_features[0]
info= all_features[1]
gamma = all_features[2]
ksi=all_features[3]
print(fisher.shape, info.shape, gamma.shape, ksi.shape)

In [ ]:
# now need to write a function that concatenates all this shit and fits a model
print gamma.shape
df_final.shape

In [ ]:
#datex=0 #which date we pick
import time
start = time.time()

for datex in range(0,10):
    print("the model date is:", list_models_dates[datex])
    fwd_dates_list_for_date = fwd_dates(list_models_dates,list_models_dates[datex]) #dates for OOS

    model_= models_list[datex] #model we will test

    loaded_models=data_cls.open_pickle_file(ticker_models_path, model_)#load models

    best_gboost=loaded_models['GBOOST']#best gradient boosting model
    best_kr = loaded_models['KR'] #best kernel ridge
    best_svc= loaded_models['SVC'] #best svc

    #######start testing
    for test_date_x in range(0,10):
        test_date = fwd_dates_list_for_date[test_date_x]
        print("you are testing on:", test_date)
        
        test_features_pickle=data_cls.ticker_features(test_date)
        test_labels = data_cls.ticker_labels_csv(date=test_date)
        #one liner converting tuple into pandas dataframe of all the features
        df_features_ = pd.concat([test_features_pickle[item]for item in range(0,4)], axis=1,sort=False) 
        #abels
        all_labels=test_labels.drop(columns=['Duration','ReturnTradedPrice','states','TradedTime','TradedPrice','ticker'], axis=1)
        idx=1
        df_labels = all_labels.iloc[:, 0:idx]
        df_concat=pd.concat([df_features_, df_labels], axis=1, sort=False)
        df= df_concat.dropna()
        label_column_loc = df.shape[1] - 1 #location of labels column
        labels = df.iloc[:, label_column_loc:label_column_loc + 1]
        features = df.drop(df.columns[label_column_loc], axis=1)
        y_predict_gboost =best_gboost.predict(features)
        y_predict_svc = best_svc.predict(features)
        y_predict_kr =best_kr.predict(features)
        print(prec_recall_report(labels,y_predict_gboost))

end = time.time()
print(end - start)

put this all below in a dictionary so you can manipulate it:
    print('ROC AUC: %.3f' % roc_auc_score(y, y_predict))
        print('Accuracy: %.2f' % accuracy_score(y, y_predict))
        print('Precision: %.3f' % precision_score(y_true=y, y_pred=y_predict))
        print('Recall: %.3f' % recall_score(y_true=y, y_pred=y_predict))
        print('F1: %.3f' % f1_score(y_true=y, y_pred=y_predict))
     

_model_results= {
    'clfs': np.empty((M, T)),
    'model_date': np.empty((M, T)),
    'data_date': np.empty((M, T)),
    'ROC': np.empty((M, T)),
    'accuracy' :np.empty((M, T))
}

##unit test for sequential prediction in sklearn###

In [ ]:
no_of_steps =features.shape[0]


In [ ]:
best_gboost.predict(features[:1])

In [ ]:
for step in range(1,5):
    y_predict=best_svc.predict(features[:step])
    print y_predict


In [ ]:
best_svc.predict(features)